In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Margin Overlap Viewer — 每个元素各自 Top 5% 版（优化版本）
------------------------------------------------
优化内容：
1. 配置集中管理
2. 增强错误处理
3. 性能优化（缓存、向量化操作）
4. 用户体验改进（进度显示、统计面板）
5. 代码结构优化
6. 每个元素独立阈值
"""

import os
import math
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from typing import Dict, List, Tuple, Optional, Set, Any
from functools import lru_cache
import logging

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib.figure import Figure
from matplotlib.ticker import MaxNLocator
from matplotlib.patches import Patch

try:
    from PIL import Image, ImageTk
except ImportError:
    Image = None

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class AppConfig:
    """应用配置集中管理"""
    CSV_PATH = "Predictions_Quebec_Area_Abitibi_18_standard_05_iter_300.csv"
    FEATURE_LIST = [
        "CO", "AU", "CU", "AG", "BA", "BAO", "BE", "BI", "CAO", "CD", "CE", "CR", "CR2O3", "CS"
    ]
    TARGET_RATIO = 0.05
    GRID_SIZE = 900
    MAX_ACTIVE = 3
    POINT_RADIUS = 1
    FIG_DPI = 100
    LEFT_PANEL_W = 320
    TOP_TOOLBAR_H = 46
    REFERENCE_IMAGE_PATH = "Quebec_Area_Abitibi_18 _INTERSECTIONS_Geopos.png"
    
    # 颜色方案
    COLOR_SCHEME = {
        'slot_0': (66, 133, 244),
        'slot_1': (219, 68, 55),
        'slot_2': (244, 180, 0),
        'overlap_01': (15, 157, 88),
        'overlap_02': (171, 71, 188),
        'overlap_12': (0, 172, 193),
        'triple': (0, 0, 0)
    }

class DataLoader:
    """数据加载器，包含错误处理"""
    
    @staticmethod
    def load_points_df(csv_path: str) -> pd.DataFrame:
        """
        加载CSV数据，包含完整的错误处理
        """
        try:
            if not os.path.exists(csv_path):
                raise FileNotFoundError(f"CSV文件不存在: {csv_path}")
                
            df = pd.read_csv(csv_path)
            
            # 验证必要列
            required_columns = ["Coord_X", "Coord_Y"]
            missing_columns = [col for col in required_columns if col not in df.columns]
            if missing_columns:
                raise ValueError(f"CSV缺少必要列: {missing_columns}")
            
            # 数据类型转换
            df["Coord_X"] = pd.to_numeric(df["Coord_X"], errors="coerce")
            df["Coord_Y"] = pd.to_numeric(df["Coord_Y"], errors="coerce")
            
            # 清理数据
            original_count = len(df)
            df = df.dropna(subset=["Coord_X", "Coord_Y"]).reset_index(drop=True)
            cleaned_count = len(df)
            
            if cleaned_count == 0:
                raise ValueError("CSV中没有有效的坐标数据")
                
            logger.info(f"数据加载成功: 原始点数 {original_count}, 有效点数 {cleaned_count}")
            
            return df
            
        except pd.errors.EmptyDataError:
            raise ValueError("CSV文件为空")
        except pd.errors.ParserError as e:
            raise ValueError(f"CSV文件解析错误: {e}")
        except Exception as e:
            raise ValueError(f"读取CSV文件失败: {e}")

class MarginCalculator:
    """边缘点计算器，包含性能优化"""
    
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self._threshold_cache: Dict[Tuple[str, float], float] = {}
        self._points_cache: Dict[Tuple[str, float], np.ndarray] = {}
    
    def _calculate_threshold(self, feature: str, target_ratio: float) -> float:
        """计算特征阈值"""
        s = pd.to_numeric(self.df[feature], errors="coerce")
        if not s.notna().any():
            return float('nan')
        
        q = 1.0 - max(0.0, min(1.0, target_ratio))
        return s.quantile(q)
    
    def get_feature_threshold(self, feature: str, target_ratio: float) -> float:
        """获取特征阈值"""
        cache_key = (feature, target_ratio)
        if cache_key not in self._threshold_cache:
            self._threshold_cache[cache_key] = self._calculate_threshold(feature, target_ratio)
        return self._threshold_cache[cache_key]
    
    def get_feature_points(self, feature: str, target_ratio: float) -> np.ndarray:
        """获取特征点数据，带缓存"""
        cache_key = (feature, target_ratio)
        if cache_key not in self._points_cache:
            threshold = self.get_feature_threshold(feature, target_ratio)
            s = pd.to_numeric(self.df[feature], errors="coerce")
            mask = (s >= threshold) & s.notna()
            pts = self.df.loc[mask, ["Coord_X", "Coord_Y"]].to_numpy(float)
            self._points_cache[cache_key] = pts
        return self._points_cache[cache_key]
    
    def build_margins_per_feature_topq(self, features: List[str], target_ratios: Dict[str, float]) -> Tuple[List[str], List[np.ndarray]]:
        """
        构建每个特征的边缘点（支持独立阈值）
        """
        # 过滤不存在的特征
        real_features = [f for f in features if f in self.df.columns]
        missing_features = [f for f in features if f not in self.df.columns]
        
        if missing_features:
            logger.warning(f"以下特征在CSV中不存在，已跳过: {missing_features}")
        
        margin_names: List[str] = []
        margins_points: List[np.ndarray] = []
        
        for feature in real_features:
            try:
                target_ratio = target_ratios.get(feature, 0.05)  # 默认5%
                pts = self.get_feature_points(feature, target_ratio)
                margin_names.append(feature)
                margins_points.append(pts)
                
                threshold = self.get_feature_threshold(feature, target_ratio)
                logger.info(f"{feature}: 阈值={threshold:.4f}, 精英点数={len(pts)} (Top {int(target_ratio*100)}%)")
                
            except Exception as e:
                logger.error(f"处理特征 {feature} 时出错: {e}")
                margin_names.append(feature)
                margins_points.append(np.empty((0, 2), float))
        
        return margin_names, margins_points

class CoordinateTransformer:
    """坐标变换器"""
    
    @staticmethod
    def make_affine_to_grid(all_pts: np.ndarray, grid_size: int):
        """创建坐标到网格的仿射变换"""
        if all_pts.size == 0:
            return lambda pts: np.empty((0, 2), np.int32), (0, 1, 0, 1)
        # old
        # X, Y = longitude, latitude = horizontal, vertical
        # x, y = all_pts[:, 0], all_pts[:, 1]
        # 改变地图显示
        # X, Y = longitude, latitude = vertical, horizontal
        x, y = all_pts[:, 1], all_pts[:, 0]
        xmin, xmax, ymin, ymax = x.min(), x.max(), y.min(), y.max()
        
        # 计算原始宽高比
        width = xmax - xmin
        height = ymax - ymin
        
        # 如果数据范围太小，设置最小范围
        if width == 0:
            width = 1.0
            xmin -= 0.5
            xmax += 0.5
        if height == 0:
            height = 1.0
            ymin -= 0.5
            ymax += 0.5
        
        # 参考世界地图比例（纬度:经度 ≈ 1:1.5），调整坐标范围以保持正方形显示
        aspect_ratio = width / height
        
        if aspect_ratio > 1.1:  # 太宽，需要增加高度
            target_height = width / 1.1
            height_expansion = (target_height - height) / 10
            ymin -= height_expansion
            ymax += height_expansion
            ymin = math.floor(ymin * 100) / 100
            ymax = math.ceil(ymax * 100) / 100
        elif aspect_ratio < 1.1:  # 太高，需要增加宽度
            target_width = height * 1.1
            width_expansion = (target_width - width) / 20 #扩展宽度
            xmin -= width_expansion
            xmax += width_expansion
            xmin = math.floor(xmin * 100) / 100
            xmax = math.ceil(xmax * 100) / 100
        xr, yr = (xmax - xmin) or 1.0, (ymax - ymin) or 1.0

        def to_px(pts_xy: np.ndarray):
            if pts_xy.size == 0:
                return np.empty((0, 2), np.int32)
            # xs = np.clip(np.rint((pts_xy[:, 0] - xmin) / xr * (grid_size - 1)), 0, grid_size - 1).astype(int)
            # # ys = np.clip(np.rint((pts_xy[:, 1] - ymin) / yr * (grid_size - 1)), 0, grid_size - 1).astype(int)
            # ys = np.clip(np.rint((ymax - pts_xy[:, 1]) / yr * (grid_size - 1)), 0, grid_size - 1).astype(int)
            # 交换坐标处理
            lats = pts_xy[:, 1]  # 纬度作为x轴
            lons = pts_xy[:, 0]  # 经度作为y轴
            
            # 水平轴：纬度 (latitude)
            xs = np.clip(np.rint((lats - xmin) / xr * (grid_size - 1)), 0, grid_size - 1).astype(int)
            # 垂直轴：经度 (longitude) - 注意这里y坐标需要翻转
            ys = np.clip(np.rint((ymax - lons) / yr * (grid_size - 1)), 0, grid_size - 1).astype(int)
            return np.stack([xs, ys], 1)
        
        
        return to_px, (xmin, xmax, ymin, ymax)

class PointRenderer:
    """点渲染器，包含性能优化"""
    
    @staticmethod
    def stamp_points_vectorized(mask: np.ndarray, xs: np.ndarray, ys: np.ndarray, r: int = 0):
        """
        使用向量化操作优化点绘制
        """
        if xs.size == 0:
            return
            
        H, W = mask.shape
        xs, ys = np.clip(xs, 0, W - 1), np.clip(ys, 0, H - 1)
        
        if r <= 0:
            mask[ys, xs] = True
            return
            
        # 使用meshgrid向量化操作
        rr = int(r)
        y_grid, x_grid = np.ogrid[-rr:rr+1, -rr:rr+1]
        disk = (x_grid**2 + y_grid**2) <= rr**2
        
        for x, y in zip(xs, ys):
            x0, x1 = max(0, x - rr), min(W, x + rr + 1)
            y0, y1 = max(0, y - rr), min(H, y + rr + 1)
            
            if x0 >= x1 or y0 >= y1:
                continue
                
            dx0, dy0 = x0 - (x - rr), y0 - (y - rr)
            dx1, dy1 = (x + rr + 1) - x1, (y + rr + 1) - y1
            
            sub_disk = disk[dy0:disk.shape[0]-dy1, dx0:disk.shape[1]-dx1]
            mask[y0:y1, x0:x1] |= sub_disk

class ThresholdManager:
    """阈值管理器"""
    
    def __init__(self, feature_list: List[str], default_ratio: float = 0.05):
        self.feature_thresholds: Dict[str, float] = {}
        for feature in feature_list:
            self.feature_thresholds[feature] = default_ratio
    
    def set_threshold(self, feature: str, ratio: float):
        """设置单个特征的阈值"""
        self.feature_thresholds[feature] = ratio
    
    def get_threshold(self, feature: str) -> float:
        """获取特征的阈值"""
        return self.feature_thresholds.get(feature, 0.05)
    
    def get_all_thresholds(self) -> Dict[str, float]:
        """获取所有特征的阈值"""
        return self.feature_thresholds.copy()
    
    def reset_all_thresholds(self, default_ratio: float = 0.05):
        """重置所有特征的阈值为默认值"""
        for feature in self.feature_thresholds:
            self.feature_thresholds[feature] = default_ratio

class StatisticsPanel:
    """统计信息面板"""
    
    def __init__(self, parent):
        self.frame = ttk.LabelFrame(parent, text="Log", padding=5)
        self.text_widget = tk.Text(
            self.frame, 
            height=8, 
            width=AppConfig.LEFT_PANEL_W//8,
            font=("Consolas", 9),
            wrap=tk.WORD
        )
        self.text_widget.pack(fill=tk.BOTH, expand=True)
        
        # 禁用编辑，但允许复制
        self.text_widget.config(state=tk.DISABLED)
    
    def pack(self, **kwargs):
        self.frame.pack(**kwargs)
    
    def update_stats_cn(self, stats_data: Dict[str, Any]):
        """更新统计信息"""
        self.text_widget.config(state=tk.NORMAL)
        self.text_widget.delete(1.0, tk.END)
        
        lines = []
        lines.append("=== 统计信息 ===")
        lines.append(f"已选择元素: {stats_data.get('selected_count', 0)}")
        lines.append(f"总点数: {stats_data.get('total_points', 0):,}")
        
        if 'overlap_info' in stats_data:
            lines.append("")
            lines.append("=== 重叠统计 ===")
            lines.extend(stats_data['overlap_info'])
        
        if 'feature_info' in stats_data:
            lines.append("")
            lines.append("=== 元素信息 ===")
            lines.extend(stats_data['feature_info'])
        
        self.text_widget.insert(1.0, "\n".join(lines))
        self.text_widget.config(state=tk.DISABLED)

    def update_stats(self, stats_data: Dict[str, Any]):
        """Update statistics information"""
        self.text_widget.config(state=tk.NORMAL)
        self.text_widget.delete(1.0, tk.END)
        
        lines = []
        lines.append("=== Statistics ===")
        lines.append(f"Selected elements: {stats_data.get('selected_count', 0)}")
        lines.append(f"Total points: {stats_data.get('total_points', 0):,}")
        
        if 'overlap_info' in stats_data:
            lines.append("")
            lines.append("=== Overlap Statistics ===")
            lines.extend(stats_data['overlap_info'])
        
        if 'feature_info' in stats_data:
            lines.append("")
            lines.append("=== Element Information ===")
            lines.extend(stats_data['feature_info'])
        
        self.text_widget.insert(1.0, "\n".join(lines))
        self.text_widget.config(state=tk.DISABLED)

class MultiAreaPointsApp:
    """主应用类（优化版本）"""
    
    def __init__(self, master, df,
                 features, target_ratio,
                 grid_size, max_active, point_radius,
                 ref_image_path: str | None = None, fig_dpi: int = 100):
        self.master = master
        self.grid_size = grid_size
        self.max_active = max_active
        self.point_radius = point_radius
        self.fig_dpi = fig_dpi
        self.df = df  # 保存原始数据框以获取坐标范围
        
        # 初始化组件
        self.chk_vars: List[tk.BooleanVar] = []
        self.chk_widgets: List[ttk.Checkbutton] = []
        self.slot_assignment: Dict[int, int] = {}
        
        # 工具类实例化
        self.calculator = MarginCalculator(df)
        self.threshold_manager = ThresholdManager(features, target_ratio)
        self.ref_img_size = self._probe_image_size(ref_image_path)
        
        # 数据加载与处理
        self._load_and_process_data()
        
        # UI构建
        self._build_ui()
        self._populate_checkboxes()
        self.update_message()
        self.update_statistics()
        self.draw()

    def _load_and_process_data(self):
        """加载和处理数据"""
        try:
            # 显示加载状态
            # self._show_loading_message("正在计算特征阈值和精英点...")
            
            # 使用优化后的计算器
            target_ratios = self.threshold_manager.get_all_thresholds()
            self.margin_names, margins_points = self.calculator.build_margins_per_feature_topq(
                AppConfig.FEATURE_LIST, target_ratios
            )
            
            if not self.margin_names:
                raise ValueError("没有找到有效的特征数据")
            
            # 坐标变换
            if any(p.size > 0 for p in margins_points):
                all_pts = np.concatenate([p for p in margins_points if p.size > 0], 0)
            else:
                all_pts = np.zeros((0, 2))
                
            to_px, self.coord_range = CoordinateTransformer.make_affine_to_grid(all_pts, self.grid_size)
            self.margins_px = [to_px(p) for p in margins_points]
            
            # 初始化槽位和颜色
            self.slot_assignment = {}
            self.slot_colors = [
                AppConfig.COLOR_SCHEME['slot_0'],
                AppConfig.COLOR_SCHEME['slot_1'], 
                AppConfig.COLOR_SCHEME['slot_2']
            ][:self.max_active]
            
            self.overlap_colors = {
                (0, 1): AppConfig.COLOR_SCHEME['overlap_01'],
                (0, 2): AppConfig.COLOR_SCHEME['overlap_02'],
                (1, 2): AppConfig.COLOR_SCHEME['overlap_12'],
            }
            self.triple_color = AppConfig.COLOR_SCHEME['triple']
            
            self.last_img_rgba: np.ndarray | None = None
            
        except Exception as e:
            logger.error(f"数据加载失败: {e}")
            messagebox.showerror("错误", f"数据加载失败: {e}")
            raise

    def _show_loading_message(self, message: str):
        """显示加载消息"""
        # if hasattr(self, 'status'):
        #     self.status.config(text=message)
        #     self.master.update()
        pass

    def _probe_image_size(self, path: str | None) -> Tuple[int, int] | None:
        """探测参考图片尺寸"""
        if not path or not os.path.exists(path) or Image is None:
            return None
        try:
            with Image.open(path) as im:
                return im.size
        except Exception as e:
            logger.warning(f"无法读取参考图片尺寸: {e}")
            return None

    # def _build_ui(self):
    #     """构建用户界面"""
    #     self.master.title("Interactive Margins")

    #     # 计算窗口尺寸
    #     if self.ref_img_size:
    #         w_px, h_px = self.ref_img_size
    #         fig_w_in, fig_h_in = w_px / self.fig_dpi, h_px / self.fig_dpi
    #     else:
    #         # 使用调整后的坐标范围计算图形尺寸
    #         xmin, xmax, ymin, ymax = self.coord_range
    #         width = xmax - xmin
    #         height = ymax - ymin
            
    #         # 根据世界地图比例调整图形尺寸
    #         if width / height > 1.5:
    #             fig_w_in = 8.0
    #             fig_h_in = fig_w_in / 1.5
    #         else:
    #             fig_h_in = 8.0
    #             fig_w_in = fig_h_in * 1.5
            
    #         w_px, h_px = int(fig_w_in * self.fig_dpi), int(fig_h_in * self.fig_dpi)

    #     win_w = w_px + AppConfig.LEFT_PANEL_W + 40
    #     win_h = h_px + AppConfig.TOP_TOOLBAR_H + 60
    #     self.master.geometry(f"{win_w}x{win_h}")

    #     # 左侧面板
    #     self.left = ttk.Frame(self.master)
    #     self.left.pack(side=tk.LEFT, fill=tk.BOTH, padx=5, pady=5)
        
    #     # 右侧面板
    #     self.right = ttk.Frame(self.master)
    #     self.right.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

    #     # 1. 标题
    #     title_label = ttk.Label(
    #         self.left,
    #         text="Features (Individual Thresholds)",
    #         font=("Arial", 14, "bold"),
    #     )
    #     title_label.pack(anchor="w", pady=(0, 10))

    #     # 2. 复选框区域（可滚动）
    #     checkbox_frame = ttk.Frame(self.left)
    #     checkbox_frame.pack(fill=tk.BOTH, expand=True, pady=(0, 10))
        
    #     canvas = tk.Canvas(checkbox_frame, highlightthickness=0)
    #     scrollbar = ttk.Scrollbar(checkbox_frame, orient="vertical", command=canvas.yview)
    #     self.scrollable_frame = ttk.Frame(canvas)
        
    #     self.scrollable_frame.bind(
    #         "<Configure>",
    #         lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
    #     )
        
    #     canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
    #     canvas.configure(yscrollcommand=scrollbar.set)
        
    #     canvas.pack(side="left", fill="both", expand=True)
    #     scrollbar.pack(side="right", fill="y")

    #     # 3. 统计信息面板
    #     self.stats_panel = StatisticsPanel(self.left)
    #     self.stats_panel.pack(fill=tk.X, pady=(0, 10))

    #     # 4. 信息显示区域
    #     self.msg = ttk.Label(
    #         self.left, 
    #         text="", 
    #         justify=tk.LEFT, 
    #         wraplength=AppConfig.LEFT_PANEL_W - 20,
    #         background="white",
    #         relief="solid",
    #         padding=(5, 5)
    #     )
    #     self.msg.pack(fill=tk.X, pady=(0, 10))

    #     # 5. 操作按钮区域
    #     btns = ttk.Frame(self.left)
    #     btns.pack(side=tk.BOTTOM, fill=tk.X, pady=(10, 0))
        
    #     ttk.Button(btns, text="Export Image", command=self.export_image).pack(side=tk.LEFT, padx=(0, 5))
    #     ttk.Button(btns, text="Clear Selection", command=self.clear_selection).pack(side=tk.LEFT)
    #     ttk.Button(btns, text="Refresh Stats", command=self.update_statistics).pack(side=tk.LEFT, padx=(5, 0))

    #     # 右侧图形区域
    #     self.fig = Figure(figsize=(fig_w_in, fig_h_in), dpi=self.fig_dpi)
    #     self.ax = self.fig.add_subplot(111)
    #     self.fig.subplots_adjust(right=0.75)
    #     self.canvas = FigureCanvasTkAgg(self.fig, master=self.right)
    #     self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    #     NavigationToolbar2Tk(self.canvas, self.right).update()

    #     self.status = ttk.Label(self.master, text="", anchor="w")
    #     self.status.pack(side=tk.BOTTOM, fill=tk.X)
    #     self.canvas.mpl_connect("motion_notify_event", self.on_motion)
    def _build_ui(self):
        """构建用户界面"""
        self.master.title("Interactive Margins")

        # 最简单直接的跨平台全屏方案
        screen_width = self.master.winfo_screenwidth()
        screen_height = self.master.winfo_screenheight()
        # 直接设置窗口尺寸为屏幕尺寸
        self.master.geometry(f"{screen_width}x{screen_height}+0+0")
        
        # 同时设置为最大化状态（双重保证）
        self.master.state('zoomed')
        
        # 强制更新
        self.master.update_idletasks()
        
        # 获取实际窗口尺寸
        actual_width = self.master.winfo_width()
        actual_height = self.master.winfo_height()
        # 计算窗口尺寸
        if self.ref_img_size:
            w_px, h_px = self.ref_img_size
            fig_w_in, fig_h_in = w_px / self.fig_dpi, h_px / self.fig_dpi
        else:
            # 使用调整后的坐标范围计算图形尺寸
            xmin, xmax, ymin, ymax = self.coord_range
            width = xmax - xmin
            height = ymax - ymin
            
            # 根据世界地图比例调整图形尺寸
            if width / height > 1.5:
                fig_w_in = 8.0
                fig_h_in = fig_w_in / 1.5
            else:
                fig_h_in = 8.0
                fig_w_in = fig_h_in * 1.5
            
            w_px, h_px = int(fig_w_in * self.fig_dpi), int(fig_h_in * self.fig_dpi)

        win_w = w_px + AppConfig.LEFT_PANEL_W + 40
        win_h = h_px + AppConfig.TOP_TOOLBAR_H + 60
        self.master.geometry(f"{win_w}x{win_h}")

        # 左侧面板
        self.left = ttk.Frame(self.master)
        self.left.pack(side=tk.LEFT, fill=tk.BOTH, padx=5, pady=5)
        
        # 右侧面板
        self.right = ttk.Frame(self.master)
        self.right.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        # 1. 标题
        title_label = ttk.Label(
            self.left,
            text="Features (Individual Thresholds)",
            font=("Arial", 14, "bold"),
        )
        title_label.pack(anchor="w", pady=(0, 10))

        # 2. 复选框区域（可滚动）
        checkbox_frame = ttk.Frame(self.left)
        checkbox_frame.pack(fill=tk.BOTH, expand=True, pady=(0, 10))
        
        canvas = tk.Canvas(checkbox_frame, highlightthickness=0)
        scrollbar = ttk.Scrollbar(checkbox_frame, orient="vertical", command=canvas.yview)
        self.scrollable_frame = ttk.Frame(canvas)
        
        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        # 3. 统计信息面板
        # self.stats_panel = StatisticsPanel(self.left)
        # self.stats_panel.pack(fill=tk.X, pady=(0, 10))

        # 4. 信息显示区域
        self.msg = ttk.Label(
            self.left, 
            text="", 
            justify=tk.LEFT, 
            wraplength=AppConfig.LEFT_PANEL_W - 20,
            background="white",
            relief="solid",
            padding=(5, 5)
        )
        self.msg.pack(fill=tk.X, pady=(0, 10))

        # 5. 操作按钮区域
        btns = ttk.Frame(self.left)
        btns.pack(side=tk.BOTTOM, fill=tk.X, pady=(10, 0))
        
        ttk.Button(btns, text="Export Image", command=self.export_image).pack(side=tk.LEFT, padx=(0, 5))
        ttk.Button(btns, text="Clear Selection", command=self.clear_selection).pack(side=tk.LEFT)
        ttk.Button(btns, text="Refresh Stats", command=self.update_statistics).pack(side=tk.LEFT, padx=(5, 0))

        # 右侧图形区域 - 重新组织布局
        right_container = ttk.Frame(self.right)
        right_container.pack(fill=tk.BOTH, expand=True)
        
        # 创建图形
        self.fig = Figure(figsize=(fig_w_in, fig_h_in), dpi=self.fig_dpi)
        self.ax = self.fig.add_subplot(111)
        self.fig.subplots_adjust(right=0.75)
        self.canvas = FigureCanvasTkAgg(self.fig, master=right_container)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        
        # 创建专门的工具栏容器，固定在底部
        toolbar_container = ttk.Frame(self.right)
        toolbar_container.pack(side=tk.BOTTOM, fill=tk.X)
        
        # 将工具栏添加到专门的容器中
        toolbar = NavigationToolbar2Tk(self.canvas, toolbar_container)
        toolbar.update()
        toolbar.pack(side=tk.BOTTOM, fill=tk.X)

        self.status = ttk.Label(self.master, text="", anchor="w")
        self.status.pack(side=tk.BOTTOM, fill=tk.X)
        self.canvas.mpl_connect("motion_notify_event", self.on_motion)
        
    def _create_individual_threshold_controls(self):
        """为每个元素创建独立的阈值控件"""
        # 清除现有的阈值控件
        for widget in self.scrollable_frame.winfo_children():
            if hasattr(widget, '_is_threshold_control'):
                widget.destroy()
        
        # 清空之前的复选框状态
        self.chk_vars.clear()
        self.chk_widgets.clear()
        
        self.threshold_vars = {}
        self.threshold_scales = {}
        self.apply_buttons = {}
        
        for i, feature in enumerate(self.margin_names):
            # 特征框架
            feature_frame = ttk.Frame(self.scrollable_frame)
            feature_frame.pack(fill=tk.X, pady=2)
            feature_frame._is_threshold_control = True
            
            # 复选框
            var = tk.BooleanVar(value=False)
            chk = ttk.Checkbutton(
                feature_frame,
                text=f"{i + 1:02d}. {feature}",
                variable=var,
                command=lambda idx=i: self.on_toggle(idx),
            )
            chk.pack(side=tk.LEFT, padx=(0, 10))
            self.chk_vars.append(var)
            self.chk_widgets.append(chk)
            
            # 阈值滑块
            threshold_var = tk.DoubleVar(value=self.threshold_manager.get_threshold(feature) * 100)
            
            # 创建滑块和标签的容器
            threshold_container = ttk.Frame(feature_frame)
            threshold_container.pack(side=tk.LEFT, fill=tk.X, expand=True)
            
            # 滑块
            scale = ttk.Scale(
                threshold_container,
                from_=1,
                to=10,
                variable=threshold_var,
                orient=tk.HORIZONTAL,
                length=80,
                command=lambda val, f=feature: self.on_individual_threshold_change(f, float(val))
            )
            scale.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
            
            # 显示当前值的标签
            value_label = ttk.Label(threshold_container, text=f"{threshold_var.get():.0f}%")
            value_label.pack(side=tk.LEFT, padx=(0, 5))
            
            # 应用按钮
            apply_btn = ttk.Button(
                feature_frame,
                text="Apply",
                width=6,
                command=lambda f=feature: self.apply_individual_threshold(f)
            )
            apply_btn.pack(side=tk.LEFT, padx=(5, 0))
            
            self.threshold_vars[feature] = threshold_var
            self.threshold_scales[feature] = (scale, value_label)
            self.apply_buttons[feature] = apply_btn
            
            # 初始状态：未勾选时禁用滑块和按钮
            scale.config(state="disabled")
            apply_btn.config(state="disabled")

    def _update_threshold_controls_state(self, index: int):
        """更新阈值控件的激活状态"""
        if index < len(self.margin_names):
            feature = self.margin_names[index]
            is_checked = self.chk_vars[index].get()
            
            if feature in self.threshold_scales:
                scale, value_label = self.threshold_scales[feature]
                if is_checked:
                    scale.config(state="normal")
                    if feature in self.apply_buttons:
                        self.apply_buttons[feature].config(state="normal")
                else:
                    scale.config(state="disabled")
                    if feature in self.apply_buttons:
                        self.apply_buttons[feature].config(state="disabled")

    def _reset_all_thresholds_to_default(self):
        """重置所有元素的阈值为5%"""
        # 重置阈值管理器中的所有阈值
        self.threshold_manager.reset_all_thresholds(AppConfig.TARGET_RATIO)
        
        # 更新所有滑块的值和显示标签
        for feature in self.threshold_vars:
            self.threshold_vars[feature].set(AppConfig.TARGET_RATIO * 100)
            if feature in self.threshold_scales:
                _, value_label = self.threshold_scales[feature]
                value_label.config(text=f"{AppConfig.TARGET_RATIO * 100:.0f}%")

    def _populate_checkboxes(self):
        """填充复选框和阈值控件"""
        self._create_individual_threshold_controls()

    def on_individual_threshold_change(self, feature: str, value: float):
        """单个阈值改变回调"""
        # 更新显示标签
        if feature in self.threshold_scales:
            _, value_label = self.threshold_scales[feature]
            value_label.config(text=f"{value:.0f}%")

    def apply_individual_threshold(self, feature: str):
        """应用单个特征的阈值"""
        try:
            new_ratio = self.threshold_vars[feature].get() / 100.0
            if new_ratio <= 0 or new_ratio > 0.5:
                messagebox.showerror("错误", "阈值必须在1%到10%之间")
                return
            
            # 更新单个特征的阈值
            self.threshold_manager.set_threshold(feature, new_ratio)
            
            # 重新加载数据
            self._reload_data()
            
        except Exception as e:
            messagebox.showerror("错误", f"应用阈值失败: {e}")

    def _reload_data(self):
        """重新加载数据（保持当前选择状态）"""
        # 保存当前选择状态（基于特征名称，而不是索引）
        current_selection_features = set()
        for i, var in enumerate(self.chk_vars):
            if var.get() and i < len(self.margin_names):
                feature_name = self.margin_names[i]
                current_selection_features.add(feature_name)
        
        # 保存槽位分配信息
        current_slot_assignment = {}
        # for slot, idx in self.slot_assignment.items():
        for idx, slot in self.slot_assignment.items():
            if idx < len(self.margin_names):
                feature_name = self.margin_names[idx]
                current_slot_assignment[feature_name] = slot
        
          
        # 重新加载数据
        self._load_and_process_data()
        
        # 重新创建阈值控件
        self._create_individual_threshold_controls()
        
        # 恢复选择状态（基于特征名称）
        for i, feature in enumerate(self.margin_names):
            if feature in current_selection_features:
                self.chk_vars[i].set(True)
                # 恢复槽位分配
                if feature in current_slot_assignment:
                    slot = current_slot_assignment[feature]
                    self.slot_assignment[i] = slot
        
        # 更新界面状态
        if len(self.slot_assignment) >= self.max_active:
            for j, w in enumerate(self.chk_widgets):
                if j not in self.slot_assignment:
                    w.state(["disabled"])
        else:
            for w in self.chk_widgets:
                w.state(["!disabled"])
        
        # 更新阈值控件状态
        for i in range(len(self.margin_names)):
            self._update_threshold_controls_state(i)
        
        self.update_message()
        self.update_statistics()
        # 强制重新绘制，确保 last_img_rgba 被更新
        self.draw()

    def on_motion(self, event):
        """鼠标移动事件"""
        pass

    def on_toggle(self, i: int):
        """复选框切换事件"""
        c = self.chk_vars[i].get()
        if c:
            if len(self.slot_assignment) >= self.max_active:
                messagebox.showwarning("Limit", f"Maximum {self.max_active} items allowed")
                self.chk_vars[i].set(False)
                return
            used = set(self.slot_assignment.values())
            free_slots = [s for s in range(self.max_active) if s not in used]
            slot = free_slots[0]
            self.slot_assignment[i] = slot
        else:
            self.slot_assignment.pop(i, None)

        # 更新界面状态
        if len(self.slot_assignment) >= self.max_active:
            for j, w in enumerate(self.chk_widgets):
                if j not in self.slot_assignment:
                    w.state(["disabled"])
        else:
            for w in self.chk_widgets:
                w.state(["!disabled"])

        # 更新阈值控件状态
        self._update_threshold_controls_state(i)

        self.update_message()
        self.update_statistics()
        self.draw()

    def clear_selection(self):
        """清除选择并重置所有阈值为5%"""
        # 重置所有复选框
        for v in self.chk_vars:
            v.set(False)
        
        # 清空槽位分配
        self.slot_assignment.clear()
        
        # 启用所有复选框
        for w in self.chk_widgets:
            w.state(["!disabled"])
        
        # 重置所有阈值为5%
        self._reset_all_thresholds_to_default()
        
        # 禁用所有阈值控件
        for i in range(len(self.margin_names)):
            self._update_threshold_controls_state(i)
        
        # 重新加载数据以应用重置的阈值
        self._reload_data()
        
        self.update_message()
        self.update_statistics()
        self.draw()

    def update_message(self):
        """更新消息显示"""
        rev = {s: i for i, s in self.slot_assignment.items()}
        lines = []
        if 0 in rev and 1 in rev:
            lines.append(f"Overlap {self.margin_names[rev[0]]}&{self.margin_names[rev[1]]}: green")
        if 0 in rev and 2 in rev:
            lines.append(f"Overlap {self.margin_names[rev[0]]}&{self.margin_names[rev[2]]}: purple")
        if 1 in rev and 2 in rev:
            lines.append(f"Overlap {self.margin_names[rev[1]]}&{self.margin_names[rev[2]]}: cyan")
        if all(k in rev for k in (0, 1, 2)):
            lines.append("Triple overlap: black")
        self.msg.config(text="\n".join(lines))

    def update_statistics(self):
        """更新统计信息"""
        rev = {s: i for i, s in self.slot_assignment.items()}
        
        stats_data = {
            'selected_count': len(rev),
            'total_points': sum(len(self.margins_px[rev[s]]) for s in rev) if rev else 0,
            'overlap_info': self._calculate_overlap_stats(rev),
            'feature_info': self._get_feature_info(rev)
        }
        
        # self.stats_panel.update_stats(stats_data)

    def _calculate_overlap_stats_cn(self, rev: Dict[int, int]) -> List[str]:
        """计算重叠统计信息"""
        lines = []
        if len(rev) >= 2:
            lines.append("重叠区域统计:")
            if len(rev) == 2:
                lines.append("  - 双击区域: 待计算")
            elif len(rev) == 3:
                lines.append("  - 三击区域: 待计算")
        return lines

    def _calculate_overlap_stats(self, rev: Dict[int, int]) -> List[str]:
        """Calculate overlap statistics"""
        lines = []
        if len(rev) >= 2:
            lines.append("Overlap Statistics:")
            if len(rev) == 2:
                lines.append("  - Double overlap: To be calculated")
            elif len(rev) == 3:
                lines.append("  - Triple overlap: To be calculated")
        return lines

    def _get_feature_info_cn(self, rev: Dict[int, int]) -> List[str]:
        """获取特征信息"""
        lines = []
        for slot, idx in sorted(rev.items()):
            if idx < len(self.margin_names):  # 确保索引有效
                feature = self.margin_names[idx]
                point_count = len(self.margins_px[idx])
                threshold_ratio = self.threshold_manager.get_threshold(feature)
                threshold_value = self.calculator.get_feature_threshold(feature, threshold_ratio)
                lines.append(f"  {feature}: {point_count}点, 阈值={threshold_value:.4f} (Top {int(threshold_ratio*100)}%)")
        return lines

    def _get_feature_info(self, rev: Dict[int, int]) -> List[str]:
        """Get feature information"""
        lines = []
        for slot, idx in sorted(rev.items()):
            if idx < len(self.margin_names):  # Ensure index is valid
                feature = self.margin_names[idx]
                point_count = len(self.margins_px[idx])
                threshold_ratio = self.threshold_manager.get_threshold(feature)
                threshold_value = self.calculator.get_feature_threshold(feature, threshold_ratio)
                lines.append(f"  {feature}: {point_count} points, threshold={threshold_value:.4f} (Top {int(threshold_ratio*100)}%)")
        return lines

    def draw(self):
        """绘制图形"""
        H = W = self.grid_size
        img = np.ones((H, W, 4), np.uint8) * 255
        masks = {s: np.zeros((H, W), bool) for s in range(self.max_active)}

        # 使用优化的点渲染
        for midx, slot in self.slot_assignment.items():
            if midx < len(self.margins_px):  # 确保索引有效
                pts = self.margins_px[midx]
                if pts.size == 0:
                    continue
                PointRenderer.stamp_points_vectorized(masks[slot], pts[:, 0], pts[:, 1], r=self.point_radius)

        # 计算各种区域
        m0, m1, m2 = masks[0], masks[1], masks[2]
        s0 = m0 & ~m1 & ~m2
        s1 = m1 & ~m0 & ~m2
        s2 = m2 & ~m0 & ~m1
        p01 = m0 & m1 & ~m2
        p02 = m0 & m2 & ~m1
        p12 = m1 & m2 & ~m0
        tri = m0 & m1 & m2

        # 应用颜色
        img[s0, :3] = self.slot_colors[0]
        img[s1, :3] = self.slot_colors[1]
        img[s2, :3] = self.slot_colors[2]
        
        if p01.any():
            img[p01, :3] = self.overlap_colors[(0, 1)]
        if p02.any():
            img[p02, :3] = self.overlap_colors[(0, 2)]
        if p12.any():
            img[p12, :3] = self.overlap_colors[(1, 2)]
        if tri.any():
            img[tri, :3] = self.triple_color

        # 关键修复：确保 last_img_rgba 总是被更新
        self.last_img_rgba = img.copy()

        # 更新图形
        self.ax.clear()
        
        # 设置坐标轴范围为真实的经纬度
        xmin, xmax, ymin, ymax = self.coord_range
        extent = [xmin, xmax, ymin, ymax]
        
        self.ax.imshow(img, origin="upper", interpolation="nearest", extent=extent)
        self.ax.set_aspect("equal")
        
        # 设置坐标轴刻度和标签
        self.ax.xaxis.set_major_locator(MaxNLocator(nbins=10))
        self.ax.yaxis.set_major_locator(MaxNLocator(nbins=10))
        
        # 添加坐标轴标签
        # self.ax.set_xlabel("Longitude")
        # self.ax.set_ylabel("Latitude")
        self.ax.set_xlabel("Latitude")  # 水平轴是纬度
        self.ax.set_ylabel("Longitude")  # 垂直轴是经度
        self.ax.grid(True, color="gray", linestyle="-", linewidth=0.5)
        
        # 显示平均阈值
        if self.slot_assignment:
            avg_threshold = np.mean([self.threshold_manager.get_threshold(self.margin_names[idx]) 
                                   for idx in self.slot_assignment.keys()])
            self.ax.set_title(f"{len(self.slot_assignment)} active features (avg top {int(avg_threshold*100)}%)")
        else:
            self.ax.set_title("No active features")

        # 更新图例
        self._update_legend()
        self.canvas.draw()

    def _update_legend(self):
        """更新图例"""
        rev = {s: i for i, s in self.slot_assignment.items()}
        handles = []
        
        for s, i in sorted(rev.items()):
            if i < len(self.margin_names):  # 确保索引有效
                handles.append(
                    Patch(
                        facecolor=tuple(c / 255 for c in self.slot_colors[s]),
                        edgecolor="black",
                        label=self.margin_names[i],
                    )
                )
        
        # 添加重叠区域图例
        # m0, m1, m2 = [i in rev.values() for i in range(3)]
        m0, m1, m2 = [s in rev for s in range(3)]
        if m0 and m1:
            handles.append(
                Patch(
                    facecolor=tuple(c / 255 for c in self.overlap_colors[(0, 1)]),
                    edgecolor="black",
                    label="Pair 01",
                )
            )
        if m0 and m2:
            handles.append(
                Patch(
                    facecolor=tuple(c / 255 for c in self.overlap_colors[(0, 2)]),
                    edgecolor="black",
                    label="Pair 02",
                )
            )
        if m1 and m2:
            handles.append(
                Patch(
                    facecolor=tuple(c / 255 for c in self.overlap_colors[(1, 2)]),
                    edgecolor="black",
                    label="Pair 12",
                )
            )
        if m0 and m1 and m2:
            handles.append(
                Patch(
                    facecolor=(0, 0, 0),
                    edgecolor="black",
                    label="Triple",
                )
            )
            
        if handles:
            self.ax.legend(
                handles=handles,
                title="Legend",
                loc="upper left",
                bbox_to_anchor=(1.02, 1),
            )

    def export_image(self):
        """导出图片"""
        if self.last_img_rgba is None:
            messagebox.showinfo("Export Image", "No image to export. Please select features first.")
            return
            
        fpath = filedialog.asksaveasfilename(
            title="Export Image",
            defaultextension=".png",
            filetypes=[
                ("PNG", "*.png"),
                ("JPEG", "*.jpg;*.jpeg"),
                ("TIFF", "*.tif;*.tiff"),
                ("All files", "*.*"),
            ],
            initialfile="margin_overlap.png",
        )
        
        if not fpath:
            return
            
        try:
            arr = self.last_img_rgba
            if Image is not None and (fpath.lower().endswith((".jpg", ".jpeg", ".tif", ".tiff"))):
                im = Image.fromarray(arr, mode="RGBA")
                if fpath.lower().endswith((".jpg", ".jpeg")):
                    im = im.convert("RGB")
                im.save(fpath)
            else:
                from matplotlib import pyplot as plt
                plt.imsave(fpath, arr)
                
            messagebox.showinfo("Export Image", f"Saved to:\\n{fpath}")
        except Exception as e:
            messagebox.showerror("Export Failed", f"Error while saving:\\n{e}")

def main():
    """主函数"""
    try:
        # 加载数据
        df = DataLoader.load_points_df(AppConfig.CSV_PATH)
        
        # 创建主窗口
        root = tk.Tk()
        # 在创建应用前立即设置窗口属性
        root.state('zoomed')  # 最大化
        root.update_idletasks()  # 强制更新
        app = MultiAreaPointsApp(
            root,
            df,
            AppConfig.FEATURE_LIST,
            AppConfig.TARGET_RATIO,
            AppConfig.GRID_SIZE,
            AppConfig.MAX_ACTIVE,
            AppConfig.POINT_RADIUS,
            ref_image_path=AppConfig.REFERENCE_IMAGE_PATH,
            fig_dpi=AppConfig.FIG_DPI,
        )
        
        # 启动主循环
        root.mainloop()
        
    except Exception as e:
        logger.error(f"应用启动失败: {e}")
        messagebox.showerror("启动错误", f"应用启动失败: {e}")

if __name__ == "__main__":
    main()

2025-11-26 12:12:35,581 - INFO - 数据加载成功: 原始点数 843780, 有效点数 843780
2025-11-26 12:12:36,369 - INFO - CO: 阈值=0.0960, 精英点数=42189 (Top 5%)
2025-11-26 12:12:36,385 - INFO - AU: 阈值=0.1110, 精英点数=42189 (Top 5%)
2025-11-26 12:12:36,397 - INFO - CU: 阈值=0.1038, 精英点数=42189 (Top 5%)
2025-11-26 12:12:36,409 - INFO - AG: 阈值=0.1789, 精英点数=42189 (Top 5%)
2025-11-26 12:12:36,424 - INFO - BA: 阈值=0.1751, 精英点数=42189 (Top 5%)
2025-11-26 12:12:36,432 - INFO - BAO: 阈值=0.0673, 精英点数=4842 (Top 5%)
2025-11-26 12:12:36,443 - INFO - BE: 阈值=0.0525, 精英点数=42189 (Top 5%)
2025-11-26 12:12:36,454 - INFO - BI: 阈值=0.1094, 精英点数=42189 (Top 5%)
2025-11-26 12:12:36,468 - INFO - CAO: 阈值=0.2558, 精英点数=42189 (Top 5%)
2025-11-26 12:12:36,477 - INFO - CD: 阈值=0.1567, 精英点数=42189 (Top 5%)
2025-11-26 12:12:36,491 - INFO - CE: 阈值=0.0801, 精英点数=42189 (Top 5%)
2025-11-26 12:12:36,504 - INFO - CR: 阈值=0.0602, 精英点数=42189 (Top 5%)
2025-11-26 12:12:36,522 - INFO - CR2O3: 阈值=0.0491, 精英点数=42189 (Top 5%)
2025-11-26 12:12:36,534 - INFO - CS: 阈值=0.1091